In [10]:
#| default_exp agents

In [11]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Agents

This module contains different types of Projective Simulation agents.

#| hide 

# Abstract Agent

In [12]:
#| export
from projective_simulation.methods.lib_helpers import CustomABCMeta
from abc import abstractmethod

class Abstract_Agent(metaclass = CustomABCMeta):
    """
    Abstract agent class any PS agent should be derived from. Asserts that the necessary methods are implemented.
    """

    def __init__(self, 
                 ECM = None, #The ECM Object to use
                 percept_processor = None, #An optional object for transforming observations prior to passing to ECM as a percept. Must have method "preprocess" 
                 action_processor = None #An optional object for transforming actions prior to passing to Environment as an actuator state. Must have method "postprocess" 
                ):
        self.ECM = ECM
        assert ECM is not None

        self.percept_processor = percept_processor
        if percept_processor is not None:
            assert hasattr(percept_processor, "preproccess")

        self.action_processor = action_processor
        if action_processor is not None:
            assert hasattr(action_processor, "postprocess")  

    @abstractmethod
    def update(self):
        ''' Updates the internal structure of the agent, typically by updating its ECM.'''
        pass
    
    @abstractmethod
    def deliberate(self):
        ''' Returns the action to be taken by the agent. Typically, this would involve getting a state from the environment, 
        passing it through the percept processor, passing it to the ECM, and then passing the output through the action processor if needed.'''
        pass
    
    

ModuleNotFoundError: No module named 'projective_simulation'

In [ ]:
#| hide

### Test ###

class test_abstract(Abstract_Agent):

    def __init__(self, ECM = 1):
        super().__init__(ECM = ECM)

    def update(self, lr = 10):
        return 0
    
    # Here we consider that we didn't implement the deliberate method
    # def deliberate(self):
    #     pass
    

try:
    agent = test_abstract()  # This should raise a TypeError
except TypeError:
    print("Got the expected TypeError, test passed.")
else:
    raise AssertionError("TestAgent() did NOT raise TypeError but it should have.") 

Got the expected TypeError, test passed.


#| hide
# Basic Agent

In [ ]:
#| export
class Basic_Agent(Abstract_Agent):
    
    def __init__(self, 
                 ECM : object, #if an ECM object is not given, num_actions must an integer
                 percept_processor : object = None # Preprocessor object for transforming observations prior to passing to ECM as a percept. Must have method "preprocess"
                ):
        """
        Simple, projective simulation (PS). Percepts are added to the ECM as new obsevations are encountered
        
        """
        self.ECM = ECM

        if percept_processor is not None:
            self.percept_processor = percept_processor
            assert hasattr(self.percept_processor, "preprocess")    
        else:
            self.percept_processor = None  

        super().__init__(ECM = self.ECM, percept_processor = percept_processor)  
        
        
    def deliberate(self, 
                observation: object # Data object passed from the environment/sensors
               )-> int : # Action to be performed.
        """
        Given an observation, returns an action. 
        """
        if self.percept_processor is not None:
            percept = self.percept_processor(observation)
        else:
            percept = observation

        action = self.ECM.sample(percept)

        return action

    def update(self, 
               reward #Value used for reinforcement. Likely a float
              ):
        """
        Given a reward, updates the ECM parameters
        """
        self.ECM.learn(reward)

In [ ]:
#| hide

from projective_simulation.ECMs import Two_Layer

In [ ]:
#| hide

agent = Basic_Agent(ECM = Two_Layer(num_actions = 2, g_damp=1, h_damp = 1))
agent.deliberate(0)

np.int64(0)

#| hide 

## Two layer Basic PS

In [ ]:
#| export
from projective_simulation.ECMs import Two_Layer

class Basic_2Layer_Agent(Basic_Agent):
    def __init__(self, 
                 num_actions : int, # The number of available actions. If ECM = is not given, must be int
                 glow: float = 0.1, # The glow (or eta) parameter. Won't be used if ECM is given
                 damp: float = 0., # The damping (or gamma) parameter. Won't be used if ECM is given
                 glow_method: str = 'sum', # The method for updating the glow. See ECMs.Two_Layer for details
                 policy : str = 'greedy', # The policy to use. See ECMs.Two_Layer for details
                 policy_parameters : dict = None, # The parameters for the policy. See ECMs.Two_Layer for details
                 percept_processor = None
                ):
        """
        Basic PS agent with a two layer ECM. Percepts are added to the ECM as new obsevations are encountered.        
        """

        self.ECM = Two_Layer(num_actions = num_actions, 
                            g_damp = glow,
                            h_damp =  damp,
                            glow_method = glow_method,
                            policy = policy, 
                            policy_parameters = policy_parameters)
        
        super().__init__(ECM = self.ECM, percept_processor = percept_processor)

In [ ]:
#| hide
Agent_2L = Basic_2Layer_Agent(num_actions = 2)
Agent_2L.deliberate(0)

np.int64(0)

#| hide
# nbdev export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()